In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_pickle("/content/drive/My Drive/Doutorado/Tese/MLData/Final/FinalDataFrameCM.pkl")

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(x='Dipole moment Z' , y='CAS Heff D', data=df, hue="Coordination number", palette="viridis", alpha=0.5, s=15)

In [ ]:
df["Coordination number"] = df["Coordination number"].apply(int)
df["Padded flatten coulumb matrix"] = df["Padded coulumb matrix"].apply(np.ndarray.flatten)

In [ ]:
# Getting rid of S = 2
df = df[df["CAS root Mult"] != 2]
df = df[df["CAS Heff E/D"] != 0]

In [ ]:
df.columns

In [ ]:
# Drop useless columns

df.drop(['Cartesian coordinates',
 'Internal coordinates',
 'Padded flatten coulumb matrix',
 'Bond lengths 1',
 'Bond lengths 2',
 'Bond angles',
 "CAS root Mult",
 'CAS GS energy (Eh)',
 'CAS transition energies',
 'CAS Kramers',
 'CAS Ms states',
 'Elements',
 'Coordinates',
 'Z',
 'Coulumb matrix',
 "SOC CAS lowest eigenv. (Eh)",
 'Padded coulumb matrix',
 'CAS 2PT D',
 'CAS 2PT E/D',
 'CAS d-orb1 (eV)',
 "CAS d-orb1 (cm-1)",
 'CAS d-orb1 (xy)',
 'CAS d-orb1 (yz)',
 'CAS d-orb1 (z2)',
 'CAS d-orb1 (xz)',
 'CAS d-orb1 (x2y2)',
 'CAS d-orb2 (eV)',
 'CAS d-orb2 (xy)',
 'CAS d-orb2 (yz)',
 'CAS d-orb2 (z2)',
 'CAS d-orb2 (xz)',
 'CAS d-orb2 (x2y2)',
 'CAS d-orb3 (eV)',
 'CAS d-orb3 (xy)',
 'CAS d-orb3 (yz)',
 'CAS d-orb3 (z2)',
 'CAS d-orb3 (xz)',
 'CAS d-orb3 (x2y2)',
 'CAS d-orb4 (eV)',
 'CAS d-orb4 (xy)',
 'CAS d-orb4 (yz)',
 'CAS d-orb4 (z2)',
 'CAS d-orb4 (xz)',
 'CAS d-orb4 (x2y2)',
 'CAS d-orb5 (eV)',
 'CAS d-orb5 (xy)',
 'CAS d-orb5 (yz)',
 'CAS d-orb5 (z2)',
 'CAS d-orb5 (xz)',
 'CAS d-orb5 (x2y2)',
 "CAS SOC a",
 "Number of elements",
 'CAS gx',
 'CAS gy',
 'CAS gz',
 'CAS giso',
 'CAS Heff E/D'], axis=1, inplace=True)

In [ ]:
df_dft = df[['Coordination number',
             'Spin deviation',
             'SOMO-LUMO gap (a.u)',
             'SOMO-1-LUMO gap',
             'SOMO-2-LUMO gap',
             'Dipole moment X',
             'Dipole moment Y',
             'Dipole moment Z',
             'CAS Heff D']]

In [ ]:
# Drop NaN values
df_dft = df_dft.dropna()

In [ ]:
df_dft

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df_dft.drop(['CAS Heff D'], axis=1).values

In [ ]:
y = df_dft[["CAS Heff D"]].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
X_train = scaler.fit_transform(X_train)

In [ ]:
len(X_test)

In [ ]:
X_test = scaler.transform(X_test)

In [ ]:
X_test[0]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Nadam

In [ ]:
early_stop = EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=100)

In [ ]:
opt = Nadam(learning_rate=0.0001)

In [ ]:
model = Sequential()

# Input layer
model.add(Dense(8, activation="selu"))

# Hidden layers
model.add(Dense(552, activation="selu"))
model.add(Dense(424, activation="selu"))
model.add(Dense(552, activation="selu"))
model.add(Dense(424, activation="selu"))
model.add(Dense(72, activation="selu"))
#model.add(Dense(82, activation="selu"))
#model.add(Dense(8, activation="selu"))


# Output layer
model.add(Dense(1))

model.compile(optimizer=opt, loss="mse")

In [ ]:
model.fit(x=X_train, y=y_train, epochs=1000, validation_data=(X_test, y_test), callbacks=[early_stop])

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses.plot()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score

In [ ]:
predictions = model.predict(X_test)

In [ ]:
predictions

In [ ]:
mean_squared_error(y_test, predictions)

In [ ]:
np.sqrt(mean_squared_error(y_test, predictions))

In [ ]:
explained_variance_score(y_test, predictions)

In [ ]:
pd.DataFrame(predictions)

In [ ]:
pd.DataFrame(y_test)

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
#model.save("/content/drive/My Drive/Doutorado/Tese/MLData/Final/KerasFinalDataFrame_927X.h5")

In [ ]:
#later_model = load_model("/content/drive/My Drive/Doutorado/Tese/MLData/Final/KerasFinalDataFrame_900_bkp.h5")

In [ ]:
later_model

In [ ]:
my_test = np.array([6, 0.003762, 0.10547099999999998, 0.13817999999999997, 0.17089499999999996, -0.03925, 0.00339, 0.0364])
#my_test2 = np.array([3, 0.004253, 	0.077239, 	0.128419, 	0.131415, 	-0.00921, 	0.86035, 	-0.03701])
#my_test3 = np.array([6, 0.006231, 0.118135, 0.118381, 0.124332, -0.003692, 0.007768, -0.009292])
#my_test4 = np.array([4, 0.008566, 0.120533, 0.123375, 0.125175, -0.26358, -0.04802, 0.55263])
#my_test5 = np.array([2, 0.01348, 0.034661, 0.052932, 0.100992, 0.0206, -0.02018, 0.09339])
#my_test6 = np.array([6, 0.002823, 0.141368, 0.157296, 0.200172, 0.17434, -0.37221, 0.72431])
my_test7 = np.array([4, 0.003354, 0.121585, 0.16092, 0.195776, 0.00577, -0.00283, -0.0033])

In [ ]:
my_test = scaler.transform(my_test.reshape(1, -1))

In [ ]:
my_test7 = scaler.transform(my_test7.reshape(1, -1))

In [ ]:
pred1 = later_model.predict(my_test)

In [ ]:
pred1

In [ ]:
pred7 = later_model.predict(my_test7)

In [ ]:
pred7

In [ ]:
X_test[3000]

In [ ]:
y_test[3000]

In [ ]:
my_test2 = np.array([4, 0.004253, 	0.077239, 	0.128419, 	0.131415, 	-0.00921, 	0.86035, 	-0.03701])

In [ ]:
    my_test2